# Chapter 15: Dividends and Fee Management

In [ ]:
# Get dividend from custodian as CSV
import pandas as pd

newDividends = pd.read_csv('/content/drive/MyDrive/Books/Robo-advisor with Python/Data/Dividends.csv')
print(newDividends)

     Account Symbol    Amount     Units
0  123456789    TLT  0.000000  0.000487
1  123456789    IEI  0.000000  0.000360
2  123456789    VTI  0.169971  0.000000
3  987654321    IEI  0.000000  0.002257
4  987654321    VTI  0.193737  0.000000


In [ ]:
accounts = [myAccount, myAccount2]

In [ ]:
for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      print(allocation.ticker + ", units: " + str(allocation.units))
    print("\n")

Account: 123456789
Cash: 1.1650591999113793
Portfolio: Moderate
IEI, units: 0.016878854867701
TLT, units: 0.009743440233
DBC, units: 0.0
GLD, units: 0.021665187975058866
VTI, units: 0.016622094138026632


Account: 987654321
Cash: 0.8349407954283179
Portfolio: Conservative
IEI, units: 0.11394768463229901
TLT, units: 0.0
DBC, units: 0.0
GLD, units: 0.019544835304941135
VTI, units: 0.019037328311973366




In [ ]:
accounts

[<__main__.Account at 0x7f72beddce90>, <__main__.Account at 0x7f72beddb8d0>]

In [ ]:
# Split/allocate back to account.goal(s).portfolio
for index, row in newDividends.iterrows():
  accountNo = row['Account']
  
  account = next((account for account in accounts if str(account.number) == str(accountNo)), None)

  if account == None:
    print("No account")
    break
  
  if row['Amount'] > 0:
    account.cashBalance += row['Amount']
  elif row['Units'] > 0:
    unitsToAllocate = row['Units']
    unitsAcrossPortfolios = 0.0

    for goal in account.goals:
      for allocation in goal.portfolio.allocations:
        if allocation.ticker == row['Symbol']:
          unitsAcrossPortfolios += allocation.units

    for goal in account.goals:
      for allocation in goal.portfolio.allocations:
        if allocation.ticker == row['Symbol']:
          allocation.units += unitsToAllocate * (allocation.units / unitsAcrossPortfolios)    

In [ ]:
for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      print(allocation.ticker + ", units: " + str(allocation.units))
    print("\n")

Account: 123456789
Cash: 1.3350306235113794
Portfolio: Moderate
IEI, units: 0.017238386524401
TLT, units: 0.0102306122447
DBC, units: 0.0
GLD, units: 0.021665187975058866
VTI, units: 0.016622094138026632


Account: 987654321
Cash: 1.028677651128318
Portfolio: Conservative
IEI, units: 0.11620468376529901
TLT, units: 0.0
DBC, units: 0.0
GLD, units: 0.019544835304941135
VTI, units: 0.019037328311973366




In [ ]:
# Create a function, standalone or account class
import pandas as pd
def allocateDividends(dividendFile: pd.DataFrame, accounts: list):

  for index, row in dividendFile.iterrows():
    accountNo = row['Account']
    
    account = next((account for account in accounts if str(account.number) == str(accountNo)), None)

    if account == None:
      print("No account")
      break
    
    if row['Amount'] > 0:
      account.cashBalance += row['Amount']
    elif row['Units'] > 0:
      unitsToAllocate = row['Units']
      unitsAcrossPortfolios = 0.0

      for goal in account.goals:
        for allocation in goal.portfolio.allocations:
          if allocation.ticker == row['Symbol']:
            unitsAcrossPortfolios += allocation.units

      for goal in account.goals:
        for allocation in goal.portfolio.allocations:
          if allocation.ticker == row['Symbol']:
            allocation.units += unitsToAllocate * (allocation.units / unitsAcrossPortfolios)    


In [ ]:
# Show (print) an account level summary of new dividends with date included
spy = yf.Ticker("SPY")
spy.dividends

Date
1993-03-19 00:00:00-05:00    0.213
1993-06-18 00:00:00-04:00    0.318
1993-09-17 00:00:00-04:00    0.286
1993-12-17 00:00:00-05:00    0.317
1994-03-18 00:00:00-05:00    0.271
                             ...  
2021-09-17 00:00:00-04:00    1.428
2021-12-17 00:00:00-05:00    1.633
2022-03-18 00:00:00-04:00    1.366
2022-06-17 00:00:00-04:00    1.577
2022-09-16 00:00:00-04:00    1.596
Name: Dividends, Length: 120, dtype: float64

In [ ]:
spy.info['yield']

0.016

In [ ]:
#Learn how to calculate Fees

In [ ]:
accounts = [myAccount, myAccount2]

In [ ]:
# Calculate AUM today
for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  aum = 0.0
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      price = float(yf.Ticker(allocation.ticker).info["previousClose"])
      aum += price * allocation.units
  print("AUM: " + '${0:.2f}'.format(aum))
  print("\n")

Account: 123456789
Cash: 1.3350306235113794
Portfolio: Moderate
AUM: $10.12
Account: 987654321
Cash: 1.028677651128318
Portfolio: Conservative
AUM: $20.69


In [ ]:
tickerString = ""
for account in accounts:
  for goal in account.goals:
    for allocation in goal.portfolio.allocations:
      if not allocation.ticker in tickerString:
        tickerString += allocation.ticker + " "
print(tickerString)

IEI TLT DBC GLD VTI 


In [ ]:
# Import the datetime module
from datetime import datetime, timedelta
from calendar import monthrange

# Get the current date
today = datetime.today()

# Get the first day of the current month
first_day_of_month = today.replace(day=1)

# Use the monthrange() method to get the number of days in the previous month
_, num_days_in_prev_month = monthrange(first_day_of_month.year, first_day_of_month.month-1)

# Subtract the number of days in the previous month from the first day of the current month to get the first day of the previous month
first_day_of_prev_month = first_day_of_month - timedelta(days=num_days_in_prev_month)

# Get the last day of the previous month by adding one day to the first day of the current month, and then subtracting one day
last_day_of_prev_month = first_day_of_month - timedelta(days=1)

# Print the first and last dates of the previous month
print(f"First date of previous month: {first_day_of_prev_month:%Y-%m-%d}")
print(f"Last date of previous month: {last_day_of_prev_month:%Y-%m-%d}")


First date of previous month: 2022-11-01
Last date of previous month: 2022-11-30


In [ ]:
# Get price history over period for all accounts and tickers (one month)
data = yf.download(tickerString, group_by="Ticker", start=first_day_of_prev_month, end=last_day_of_prev_month)
data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
data = data.dropna()
data.columns = data.columns.droplevel(1)
print(data)

[*********************100%***********************]  5 of 5 completed
                   IEI         TLT        DBC         GLD         VTI
Date                                                                 
2022-11-01  113.209999   96.769997  25.500000  153.460007  193.429993
2022-11-02  113.110001   96.349998  25.730000  152.389999  188.339996
2022-11-03  112.750000   95.830002  25.709999  151.809998  186.539993
2022-11-04  112.839996   94.220001  26.580000  156.470001  188.940002
2022-11-07  112.610001   93.279999  26.350000  155.850006  190.660004
2022-11-08  113.010002   94.300003  26.030001  159.449997  191.660004
2022-11-09  113.360001   94.610001  25.430000  158.649994  187.529999
2022-11-10  114.989998   98.250000  25.660000  163.479996  198.139999
2022-11-11  114.849998   97.889999  26.150000  164.559998  200.160004
2022-11-14  114.669998   97.650002  25.760000  164.919998  198.350006
2022-11-15  115.180000   99.230003  26.059999  165.500000  200.360001
2022-11-16  115.52999

In [ ]:
# Calculate average AUM over period
for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  aum = 0.0
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      for index, row in data.iterrows():
        price = row[allocation.ticker]
        aum += price * allocation.units
  aum = aum / len(data)
  print("Average AUM: " + '${0:.2f}'.format(aum))
  print("\n")

Account: 123456789
Cash: 1.3350306235113794
Portfolio: Moderate
Average AUM: $9.73


Account: 987654321
Cash: 1.028677651128318
Portfolio: Conservative
Average AUM: $20.17




In [ ]:
# Calculate fee
feeRate = 0.001 # 10bps

for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  aum = 0.0
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      for index, row in data.iterrows():
        price = row[allocation.ticker]
        aum += price * allocation.units
  aum = aum / len(data)
  fee = aum * feeRate * (num_days_in_prev_month / 365)
  print("Average AUM: " + '${0:.2f}'.format(aum))
  print("Fee: $" + str(fee))
  print("\n")

Account: 123456789
Cash: 1.3350306235113794
Portfolio: Moderate
Average AUM: $9.73
Fee: $0.0007996344333910244


Account: 987654321
Cash: 1.028677651128318
Portfolio: Conservative
Average AUM: $20.17
Fee: $0.001658052571063181




In [ ]:
# May need to create sell order(s) if cash < AUM

In [ ]:
# Generate broker instructions for fees (CSV?)
feeRate = 0.001 # 10bps
feeTable = pd.DataFrame(columns=['Account','FeeAmount'])

for account in accounts:
  aum = 0.0
  for goal in account.goals:
    for allocation in goal.portfolio.allocations:
      for index, row in data.iterrows():
        price = row[allocation.ticker]
        aum += price * allocation.units
  aum = aum / len(data)
  fee = aum * feeRate * (num_days_in_prev_month / 365)
  new_row = {'Account':account.number, 'FeeAmount':fee}
  feeTable = feeTable.append(new_row, ignore_index=True)

print(feeTable)

     Account  FeeAmount
0  123456789   0.000800
1  987654321   0.001658


In [ ]:
pd.set_option("display.precision", 8)

In [ ]:
# Generate broker instructions for fees (CSV?)
feeRate = 0.001 # 10bps
feeTable = pd.DataFrame(columns=['Account','FeeAmount'])

for account in accounts:
  aum = 0.0
  for goal in account.goals:
    for allocation in goal.portfolio.allocations:
      for index, row in data.iterrows():
        price = row[allocation.ticker]
        aum += price * allocation.units
  aum = aum / len(data)
  fee = aum * feeRate * (num_days_in_prev_month / 365)
  new_row = {'Account':account.number, 'FeeAmount':fee}
  feeTable = feeTable.append(new_row, ignore_index=True)

print(feeTable)

     Account   FeeAmount
0  123456789  0.00079963
1  987654321  0.00165805


In [ ]:
# Create function
def generateFeeInstructions(accounts: list) -> pd.DataFrame:
  tickerString = ""
  for account in accounts:
    for goal in account.goals:
      for allocation in goal.portfolio.allocations:
        if not allocation.ticker in tickerString:
          tickerString += allocation.ticker + " "
  
  from datetime import datetime, timedelta
  from calendar import monthrange
  today = datetime.today()
  first_day_of_month = today.replace(day=1)
  _, num_days_in_prev_month = monthrange(first_day_of_month.year, first_day_of_month.month-1)
  first_day_of_prev_month = first_day_of_month - timedelta(days=num_days_in_prev_month)
  last_day_of_prev_month = first_day_of_month - timedelta(days=1)

  data = yf.download(tickerString, group_by="Ticker", start=first_day_of_prev_month, end=last_day_of_prev_month)
  data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
  data = data.dropna()
  data.columns = data.columns.droplevel(1)

  feeRate = 0.001 # 10bps
  feeTable = pd.DataFrame(columns=['Account','FeeAmount'])

  for account in accounts:
    aum = 0.0
    for goal in account.goals:
      for allocation in goal.portfolio.allocations:
        for index, row in data.iterrows():
          price = row[allocation.ticker]
          aum += price * allocation.units
    aum = aum / len(data)
    fee = aum * feeRate * (num_days_in_prev_month / 365)
    new_row = {'Account':account.number, 'FeeAmount':fee}
    feeTable = feeTable.append(new_row, ignore_index=True)

  return feeTable

In [ ]:
# Save to CSV
feeFile = generateFeeInstructions(accounts)

[*********************100%***********************]  5 of 5 completed


In [ ]:
feeFile.to_csv('/content/drive/MyDrive/Books/Robo-advisor with Python/Data/Fees.csv')